# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_data = pd.read_csv("../WeatherPy/weather_df.csv")
weather_data

,Unnamed: 0,City,Latitude,Longitude,Max Temp.,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,lubumbashi,-11.6609,27.4794,64.06,97,100,4.34,CD,1612915772
1,1,constitucion,-35.3333,-72.4167,61.48,87,10,2.89,CL,1612915256
2,2,comodoro rivadavia,-45.8667,-67.5000,75.20,53,0,6.91,AR,1612915713
3,3,butaritari,3.0707,172.7902,81.37,82,50,14.32,KI,1612915772
4,4,omboue,-1.5746,9.2618,77.49,89,91,2.68,GA,1612915773
...,...,...,...,...,...,...,...,...,...,...
541,541,sao miguel do araguaia,-13.2750,-50.1628,72.68,96,33,3.29,BR,1612915864
542,542,taloqan,36.7361,69.5345,45.82,57,78,3.91,AF,1612915864
543,543,paso de carrasco,-34.8603,-56.0522,71.60,73,75,18.41,UY,1612915865
544,544,akonolinga,3.7667,12.2500,69.40,92,89,2.73,CM,1612915865


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#Configure gmaps
gmaps.configure(api_key=g_key)

#Store lat and lng as locations
locations = weather_data[["Latitude", "Longitude"]]

#Store humidity as weight
weights = weather_data["Humidity"]

In [4]:
#Plot Heatmap
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=weights, dissipating=False,
                                max_intensity=100, point_radius = 5)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
vacation_cities = weather_data.loc[(weather_data["Wind Speed"] <= 10) & (weather_data["Cloudiness"] == 0) & 
                                  (weather_data["Max Temp."] >= 70) & (weather_data["Max Temp."] <= 80)].dropna()  
vacation_cities

,Unnamed: 0,City,Latitude,Longitude,Max Temp.,Humidity,Cloudiness,Wind Speed,Country,Date
2,2,comodoro rivadavia,-45.8667,-67.5000,75.20,53,0,6.91,AR,1612915713
18,18,saint-louis,16.3333,-15.0000,77.00,36,0,2.30,SN,1612915774
40,40,east london,-33.0153,27.9116,70.57,82,0,2.06,ZA,1612915705
91,91,coihaique,-45.5752,-72.0662,78.80,38,0,3.58,CL,1612915784
150,150,saint-philippe,-21.3585,55.7679,78.80,65,0,2.30,RE,1612915736
189,189,epe,6.5841,3.9834,79.41,89,0,3.49,NG,1612915797
196,196,srivardhan,18.0333,73.0167,70.61,37,0,4.94,IN,1612915797
255,255,carahue,-38.7000,-73.1667,71.60,64,0,6.91,CL,1612915813
266,266,codrington,-38.2667,141.9667,75.00,48,0,5.44,AU,1612915496
381,381,oussouye,12.4850,-16.5469,77.00,73,0,4.61,SN,1612915836


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df = vacation_cities.loc[:, ["City", "Country", "Latitude", "Longitude"]]
hotel_df["Hotel Name"] = ""

hotel_df

,City,Country,Latitude,Longitude,Hotel Name
2,comodoro rivadavia,AR,-45.8667,-67.5000,
18,saint-louis,SN,16.3333,-15.0000,
40,east london,ZA,-33.0153,27.9116,
91,coihaique,CL,-45.5752,-72.0662,
150,saint-philippe,RE,-21.3585,55.7679,
189,epe,NG,6.5841,3.9834,
196,srivardhan,IN,18.0333,73.0167,
255,carahue,CL,-38.7000,-73.1667,
266,codrington,AU,-38.2667,141.9667,
381,oussouye,SN,12.4850,-16.5469,


In [7]:
target_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {"type": "hotel",
         "keyword": "hotel",
         "radius": 5000,
         "key": g_key}

In [8]:
for index, row in hotel_df.iterrows():
    lat = row["Latitude"]
    lng = row["Longitude"]
    city = row["City"]
    
    params["location"] = f"{lat},{lng}"
    
    print(f"Retrieving Results for Index {index}: {city}.")
    response = requests.get(target_url, params=params).json()
    results = response["results"]
    
    try:
        print(f"Closest hotel in {city}: {results[0]['name']}")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']
    except (KeyError, IndexError):
        print(f'Missing Result...skipping')
        
 

Retrieving Results for Index 2: comodoro rivadavia.
Closest hotel in comodoro rivadavia: Lucania Palazzo Hotel
Retrieving Results for Index 18: saint-louis.
Missing Result...skipping
Retrieving Results for Index 40: east london.
Closest hotel in east london: Blue Lagoon Hotel
Retrieving Results for Index 91: coihaique.
Closest hotel in coihaique: Patagonia House
Retrieving Results for Index 150: saint-philippe.
Closest hotel in saint-philippe: Les Embruns Du Baril
Retrieving Results for Index 189: epe.
Closest hotel in epe: Procare Suites & Resort Ltd
Retrieving Results for Index 196: srivardhan.
Closest hotel in srivardhan: Tranquil Beach Resort
Retrieving Results for Index 255: carahue.
Closest hotel in carahue: Hostal de carahue
Retrieving Results for Index 266: codrington.
Closest hotel in codrington: Codrington Gardens Bed And Breakfast
Retrieving Results for Index 381: oussouye.
Closest hotel in oussouye: Campement ALULUM
Retrieving Results for Index 387: tomatlan.
Closest hotel 

In [9]:
hotel_df

,City,Country,Latitude,Longitude,Hotel Name
2,comodoro rivadavia,AR,-45.8667,-67.5000,Lucania Palazzo Hotel
18,saint-louis,SN,16.3333,-15.0000,
40,east london,ZA,-33.0153,27.9116,Blue Lagoon Hotel
91,coihaique,CL,-45.5752,-72.0662,Patagonia House
150,saint-philippe,RE,-21.3585,55.7679,Les Embruns Du Baril
189,epe,NG,6.5841,3.9834,Procare Suites & Resort Ltd
196,srivardhan,IN,18.0333,73.0167,Tranquil Beach Resort
255,carahue,CL,-38.7000,-73.1667,Hostal de carahue
266,codrington,AU,-38.2667,141.9667,Codrington Gardens Bed And Breakfast
381,oussouye,SN,12.4850,-16.5469,Campement ALULUM


In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [11]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)


fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))